In [1]:
# Paquetes
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta
import dateutil.relativedelta
import gc

pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',400)

# Cargando TU

In [2]:
# Cargando base final

path = 's3://adl-refined-dev-popular/parquet/TC_adquisicion/muestra_perfiladosPiloto'

data_final = pd.read_parquet(path, engine='pyarrow')
print(data_final.shape)
data_final.head()

(13565, 14)


,TIPO_IDENTIFICACION,NUMERO_IDENTIFICACION,NOMBRE_CAMPAÑA,FECHA_INICIO,FECHA_FIN,TIPO,CIUDAD_CRM,ID_CLIENTE,ID_NUM_IDENTIFICACION,SCORE,DECIL,GRUPO_PROB,GRUPOS,SEGMENTO
0,1,10001108,Act_02_19_202007,44044,44135,CLIENTE ACTIVO,BOGOTA,205552310859223401,10001108,0.000330,2.0,grupo_prob_alta,grupo_mkd,seg_perfilado
1,1,10003005,Act_02_19_202007,44044,44135,CLIENTE ACTIVO,GIRARDOT,767952295444029901,10003005,0.001942,1.0,grupo_prob_alta,grupo_mkd,seg_perfilado
2,1,10011983,Act_02_19_202007,44044,44135,CLIENTE ACTIVO,CALI,521152307006793601,10011983,0.000021,5.0,grupo_prob_baja,grupo_mkd,seg_perfilado
3,1,10012781,Act_02_19_202007,44044,44135,CLIENTE ACTIVO,BOGOTA,158852311657703101,10012781,0.000099,3.0,grupo_prob_alta,grupo_mkd,seg_perfilado
4,1,10012969,Act_02_19_202007,44044,44135,CLIENTE ACTIVO,MEDELLIN,797552318205826201,10012969,0.000557,2.0,grupo_prob_alta,grupo_mkd,seg_perfilado


In [3]:
data_final.dtypes

TIPO_IDENTIFICACION        int64
NUMERO_IDENTIFICACION     object
NOMBRE_CAMPAÑA            object
FECHA_INICIO               int64
FECHA_FIN                  int64
TIPO                      object
CIUDAD_CRM                object
ID_CLIENTE                object
ID_NUM_IDENTIFICACION     object
SCORE                    float32
DECIL                    float64
GRUPO_PROB                object
GRUPOS                    object
SEGMENTO                  object
dtype: object

In [4]:
data_final.columns = data_final.columns.str.lower()

data_final['id_cliente'] = data_final['id_cliente'].astype('int')

# Cargando info modelo banco

## Cargando info libranzas

In [5]:
# Cargando base final

path = 's3://adl-refined-dev-popular/parquet/TC_adquisicion/base_libranzas_202001-202007'

data_lib = pd.read_parquet(path, engine='pyarrow')
print(data_lib.shape)
data_lib.head()

(5482702, 19)


,id_cliente,num_lib_solicitadas,prom_monto_solicitado,prom_monto_aprobado,prom_monto_desembolsado,prom_monto_novado,min_anos_ult_lib,prom_prop_desembolso,prom_n_cuotas,prom_valor_cuota,prom_tasa,prom_dias_procesamiento,max_dias_procesamiento,n_novaciones,sector_ultlibranza,subsector_ultlibranza,tipo_pagaduria_ultlibranza,periodo,fecha_lib
0,101052294884460201,2,7050000.0,6385000.0,5982935.0,0.0,11.508381,0.934040,72.0,164258.50,23.610001,3.50,6.0,0,PENSIONADOS,Fopep,NACIONAL CENTRALIZADA,2020-01,202001
1,101052314032556001,1,70000000.0,70000000.0,69990040.0,0.0,1.242628,0.999858,72.0,1418040.00,10.800000,1.00,1.0,0,PENSIONADOS,Fiduprevisora,NACIONAL CENTRALIZADA,2020-01,202001
2,101055260548785601,4,16175000.0,16175000.0,5261897.0,10197711.0,5.387833,0.402476,63.0,365051.75,13.590000,3.75,7.0,3,ORDEN NACIONAL,Policia Nacional,NACIONAL CENTRALIZADA,2020-01,202001
3,101055260650910902,4,12125000.0,12125000.0,7560881.0,4087683.5,4.807011,0.602487,60.0,279657.25,15.000000,3.25,7.0,3,ORDEN NACIONAL,Policia Nacional,NACIONAL CENTRALIZADA,2020-01,202001
4,101055260683554401,2,18000000.0,18000000.0,14080672.0,3618687.5,2.686464,0.831607,60.0,436291.00,15.480000,6.50,9.0,1,ORDEN NACIONAL,Policia Nacional,NACIONAL CENTRALIZADA,2020-01,202001


In [6]:
data_lib.dtypes

id_cliente                      int64
num_lib_solicitadas             int64
prom_monto_solicitado         float32
prom_monto_aprobado           float32
prom_monto_desembolsado       float32
prom_monto_novado             float32
min_anos_ult_lib              float64
prom_prop_desembolso          float32
prom_n_cuotas                 float64
prom_valor_cuota              float32
prom_tasa                     float32
prom_dias_procesamiento       float64
max_dias_procesamiento        float64
n_novaciones                    int64
sector_ultlibranza             object
subsector_ultlibranza          object
tipo_pagaduria_ultlibranza     object
periodo                        object
fecha_lib                       int64
dtype: object

In [7]:
data_lib = data_lib[data_lib['fecha_lib'] == 202007]

## Cargando info pasivo

In [8]:
# Cargando base final

path = 's3://adl-refined-dev-popular/parquet/TC_adquisicion/pasivo_201801_202006'

data_pasv = pd.read_parquet(path, engine='pyarrow')
print(data_pasv.shape)
data_pasv.head()

(21400337, 7)


,id_cliente,fecha_pasivo,pasv_antig_total,pasv_num_meses_ult_apertura,pasv_dias_desde_ultima_trans,pasv_saldo_total_1mes_atras,pasv_saldo_ca_1mes_atras
0,101052294083942902,201801,314.766667,141.800000,6.0,56.212330,0.0
1,101052294083942902,201802,315.800000,142.833333,9.0,46.309361,0.0
2,101052294083942902,201803,316.733333,143.766667,5.0,113.393348,0.0
3,101052294083942902,201804,317.766667,144.800000,6.0,102.461171,0.0
4,101052294083942902,201805,318.766667,0.966667,2.0,19.947436,0.0


In [9]:
data_pasv.dtypes

id_cliente                        int64
fecha_pasivo                      int64
pasv_antig_total                float64
pasv_num_meses_ult_apertura     float64
pasv_dias_desde_ultima_trans    float64
pasv_saldo_total_1mes_atras     float64
pasv_saldo_ca_1mes_atras        float64
dtype: object

In [10]:
data_pasv = data_pasv[data_pasv['fecha_pasivo']==202006]

## Cargando info TX Pasivo

In [11]:
path = 's3://adl-refined-dev-popular/parquet/TC_adquisicion/tx_pasivo_202006_vars_finales_model_ambos2'

data_txpasv = pd.read_parquet(path, engine='pyarrow')
print(data_txpasv.shape)
data_txpasv.head()

(3935446, 14)


,id_cliente,PERIODO,num_tx_ult_1mes,num_tx_ult_2meses,num_tx_ult_6meses,sum_tx_linea,sum_tx_nolinea,fecha_ult_tx,dias_desde_ult_tx,num_tx_ult_1mes_ahr,num_tx_ult_4meses_ahr,sum_tx_linea_ahr,sum_tx_nolinea_ahr,dias_desde_ult_tx_ahr
0,101055266603150201,202002,0.000000,1.000000,4.000000,2.0,NaN,2020-02-01,28,0.000000,4.000000,2.0,NaN,28.0
1,101958376376088702,202006,1.000000,1.000000,1.000000,1.0,NaN,2020-06-02,28,1.000000,1.000000,1.0,NaN,28.0
2,102255266234000802,202003,9.571429,21.285714,36.857143,2.0,NaN,2020-03-31,0,9.571429,36.857143,2.0,NaN,0.0
3,102255724085482901,202002,0.000000,8.000000,22.000000,6.0,NaN,2020-02-28,1,0.000000,22.000000,6.0,NaN,1.0
4,102455266339841101,201912,3.000000,3.000000,3.000000,8.0,NaN,2019-12-31,0,3.000000,3.000000,8.0,NaN,0.0


In [12]:
# # Cargando base final

# path = "s3://adl-refined-dev-popular/parquet/TC_adquisicion/tx_pasivo_202006_vars_finales_model"

# data_txpasv = pd.read_parquet(path, engine='pyarrow')
# print(data_txpasv.shape)
# data_txpasv.head()

In [13]:
data_txpasv['PERIODO'].value_counts()

201912    598226
202001    580107
202003    568964
202002    567937
202006    547059
202005    537717
202004    535436
Name: PERIODO, dtype: int64

In [14]:
data_txpasv.dtypes

id_cliente                object
PERIODO                   object
num_tx_ult_1mes          float64
num_tx_ult_2meses        float64
num_tx_ult_6meses        float64
sum_tx_linea             float64
sum_tx_nolinea           float64
fecha_ult_tx              object
dias_desde_ult_tx          int32
num_tx_ult_1mes_ahr      float64
num_tx_ult_4meses_ahr    float64
sum_tx_linea_ahr         float64
sum_tx_nolinea_ahr       float64
dias_desde_ult_tx_ahr    float64
dtype: object

In [15]:
data_txpasv = data_txpasv[data_txpasv['PERIODO']=='202006']

In [16]:
data_txpasv = data_txpasv.rename(columns={'num_tx_ult_1mes':'num_tx_ult_1mes_pasv','num_tx_ult_2meses':'num_tx_ult_2meses_pasv','num_tx_ult_6meses':'num_tx_ult_6meses_pasv',
                                          'sum_tx_linea':'sum_tx_linea_pasv','sum_tx_nolinea':'sum_tx_nolinea_pasv','dias_desde_ult_tx':'dias_desde_ult_tx_pasv'})
data_txpasv['id_cliente'] = data_txpasv['id_cliente'].astype('int')

In [17]:
data_txpasv.head()

,id_cliente,PERIODO,num_tx_ult_1mes_pasv,num_tx_ult_2meses_pasv,num_tx_ult_6meses_pasv,sum_tx_linea_pasv,sum_tx_nolinea_pasv,fecha_ult_tx,dias_desde_ult_tx_pasv,num_tx_ult_1mes_ahr,num_tx_ult_4meses_ahr,sum_tx_linea_ahr,sum_tx_nolinea_ahr,dias_desde_ult_tx_ahr
1,101958376376088702,202006,1.000000,1.000000,1.000000,1.0,NaN,2020-06-02,28,1.000000,1.000000,1.0,NaN,28.0
8,102652294123116001,202006,2.333333,2.666667,2.666667,NaN,3.0,2020-06-19,11,2.333333,2.666667,NaN,3.0,11.0
14,102652294793663401,202006,2.666667,4.666667,19.333333,3.0,NaN,2020-06-26,4,2.666667,12.333333,3.0,NaN,4.0
20,102652295678115801,202006,1.000000,2.000000,6.000000,1.0,NaN,2020-06-26,4,1.000000,4.000000,1.0,NaN,4.0
25,102652296335428801,202006,3.000000,5.800000,14.800000,5.0,NaN,2020-06-19,11,3.000000,9.800000,5.0,NaN,11.0


In [18]:
data_finalF = pd.merge(data_final,data_lib, how='left')
data_finalF = pd.merge(data_finalF,data_pasv, how='left')
data_finalF = pd.merge(data_finalF,data_txpasv, how='left')

print(data_final.shape)
print(data_finalF.shape)

data_finalF.head()

(13565, 14)
(13565, 51)


,tipo_identificacion,numero_identificacion,nombre_campaña,fecha_inicio,fecha_fin,tipo,ciudad_crm,id_cliente,id_num_identificacion,score,decil,grupo_prob,grupos,segmento,num_lib_solicitadas,prom_monto_solicitado,prom_monto_aprobado,prom_monto_desembolsado,prom_monto_novado,min_anos_ult_lib,prom_prop_desembolso,prom_n_cuotas,prom_valor_cuota,prom_tasa,prom_dias_procesamiento,max_dias_procesamiento,n_novaciones,sector_ultlibranza,subsector_ultlibranza,tipo_pagaduria_ultlibranza,periodo,fecha_lib,fecha_pasivo,pasv_antig_total,pasv_num_meses_ult_apertura,pasv_dias_desde_ultima_trans,pasv_saldo_total_1mes_atras,pasv_saldo_ca_1mes_atras,PERIODO,num_tx_ult_1mes_pasv,num_tx_ult_2meses_pasv,num_tx_ult_6meses_pasv,sum_tx_linea_pasv,sum_tx_nolinea_pasv,fecha_ult_tx,dias_desde_ult_tx_pasv,num_tx_ult_1mes_ahr,num_tx_ult_4meses_ahr,sum_tx_linea_ahr,sum_tx_nolinea_ahr,dias_desde_ult_tx_ahr
0,1,10001108,Act_02_19_202007,44044,44135,CLIENTE ACTIVO,BOGOTA,205552310859223401,10001108,0.000330,2.0,grupo_prob_alta,grupo_mkd,seg_perfilado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,10003005,Act_02_19_202007,44044,44135,CLIENTE ACTIVO,GIRARDOT,767952295444029901,10003005,0.001942,1.0,grupo_prob_alta,grupo_mkd,seg_perfilado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,202006.0,153.266667,9.733333,11.0,71.567122,71.567122,202006,13.368421,28.210526,168.473684,17.0,NaN,2020-06-27,3.0,13.368421,76.947368,17.0,NaN,3.0
2,1,10011983,Act_02_19_202007,44044,44135,CLIENTE ACTIVO,CALI,521152307006793601,10011983,0.000021,5.0,grupo_prob_baja,grupo_mkd,seg_perfilado,1.0,1000000.0,1000000.0,922319.0,0.0,7.82345,0.922319,6.0,173677.0,14.28,1.0,1.0,0.0,ORDEN NACIONAL,Policia Nacional,NACIONAL CENTRALIZADA,2020-07,202007.0,202006.0,193.466667,193.466667,8.0,1.042724,1.042724,202006,10.250000,19.250000,73.000000,16.0,NaN,2020-06-30,0.0,10.250000,42.250000,16.0,NaN,0.0
3,1,10012781,Act_02_19_202007,44044,44135,CLIENTE ACTIVO,BOGOTA,158852311657703101,10012781,0.000099,3.0,grupo_prob_alta,grupo_mkd,seg_perfilado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,202006.0,78.600000,78.600000,8.0,0.088371,0.088371,202006,10.388889,19.333333,63.666667,18.0,NaN,2020-06-27,3.0,10.388889,38.722222,18.0,NaN,3.0
4,1,10012969,Act_02_19_202007,44044,44135,CLIENTE ACTIVO,MEDELLIN,797552318205826201,10012969,0.000557,2.0,grupo_prob_alta,grupo_mkd,seg_perfilado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
# ## Set index

# data_final.set_index('id_cliente',inplace=True)

# data_lib.set_index('id_cliente',inplace=True)
# data_pasv.set_index('id_cliente',inplace=True)
# data_txpasv.set_index('id_cliente',inplace=True)

# data_finalF = data_final.join(data_lib, how='left').join(data_pasv, how='left').join(data_txpasv, how='left')

# print(data_final.shape)
# print(data_finalF.shape)

# data_finalF.head()

In [20]:
data_finalF.count()/13565*100

tipo_identificacion             100.000000
numero_identificacion           100.000000
nombre_campaña                  100.000000
fecha_inicio                    100.000000
fecha_fin                       100.000000
tipo                            100.000000
ciudad_crm                      100.000000
id_cliente                      100.000000
id_num_identificacion           100.000000
score                           100.000000
decil                           100.000000
grupo_prob                      100.000000
grupos                          100.000000
segmento                        100.000000
num_lib_solicitadas               9.598231
prom_monto_solicitado             9.598231
prom_monto_aprobado               9.598231
prom_monto_desembolsado           9.598231
prom_monto_novado                 9.598231
min_anos_ult_lib                  9.598231
prom_prop_desembolso              9.598231
prom_n_cuotas                     9.598231
prom_valor_cuota                  9.598231
prom_tasa  

## Cargando Modelo banco

In [21]:
import pickle
## Cargando Modelo

fecha = '20200623' # definir fecha
xgboost_filename = '/mnt/work/CU_adquisicionTC/Notebooks/creacion_modelo/Modelos PKL/Model_perf_'+fecha+'_VFinal.pkl'
xgboost_pkl = open(xgboost_filename, 'rb')
model = pickle.load(xgboost_pkl)


/home/jupyter/anaconda3/lib/python3.7/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.23.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [22]:
columns = ['sum_tx_linea_pasv','pasv_antig_total','dias_desde_ult_tx_pasv',
    'pasv_num_meses_ult_apertura','pasv_dias_desde_ultima_trans','num_tx_ult_4meses_ahr','pasv_saldo_total_1mes_atras',
           'num_tx_ult_2meses_pasv','num_tx_ult_1mes_pasv','min_anos_ult_lib',
           'prom_monto_desembolsado','sum_tx_nolinea_ahr','num_tx_ult_6meses_pasv','prom_tasa']

In [23]:
data_finalF[columns].head()

,sum_tx_linea_pasv,pasv_antig_total,dias_desde_ult_tx_pasv,pasv_num_meses_ult_apertura,pasv_dias_desde_ultima_trans,num_tx_ult_4meses_ahr,pasv_saldo_total_1mes_atras,num_tx_ult_2meses_pasv,num_tx_ult_1mes_pasv,min_anos_ult_lib,prom_monto_desembolsado,sum_tx_nolinea_ahr,num_tx_ult_6meses_pasv,prom_tasa
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,17.0,153.266667,3.0,9.733333,11.0,76.947368,71.567122,28.210526,13.368421,NaN,NaN,NaN,168.473684,NaN
2,16.0,193.466667,0.0,193.466667,8.0,42.250000,1.042724,19.250000,10.250000,7.82345,922319.0,NaN,73.000000,14.28
3,18.0,78.600000,3.0,78.600000,8.0,38.722222,0.088371,19.333333,10.388889,NaN,NaN,NaN,63.666667,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
# Score info banco

data_finalF['score_info_banco'] = model.predict_proba(data_finalF[columns])[:, 1]


In [25]:
# decil Test
data_finalF['decil_info_banco'] = np.where(data_finalF['score_info_banco'] <= 0.000159, 1,
                                    np.where(data_finalF['score_info_banco'] <= 0.000282, 2,
                                            np.where(data_finalF['score_info_banco'] <= 0.000636, 3,
                                                    np.where(data_finalF['score_info_banco'] <= 0.001883, 4, 5))))

In [26]:
# Grupos Probabilidad
data_finalF['grupo_prob_info_banco'] = np.where(data_finalF['decil_info_banco'] >= 3, 'grupo_prob_alta', 'grupo_prob_baja')

In [27]:
data_finalF['decil_TU'] = [6-x for x in data_finalF['decil'] if x>0] 

In [28]:
data_finalF.head()

,tipo_identificacion,numero_identificacion,nombre_campaña,fecha_inicio,fecha_fin,tipo,ciudad_crm,id_cliente,id_num_identificacion,score,decil,grupo_prob,grupos,segmento,num_lib_solicitadas,prom_monto_solicitado,prom_monto_aprobado,prom_monto_desembolsado,prom_monto_novado,min_anos_ult_lib,prom_prop_desembolso,prom_n_cuotas,prom_valor_cuota,prom_tasa,prom_dias_procesamiento,max_dias_procesamiento,n_novaciones,sector_ultlibranza,subsector_ultlibranza,tipo_pagaduria_ultlibranza,periodo,fecha_lib,fecha_pasivo,pasv_antig_total,pasv_num_meses_ult_apertura,pasv_dias_desde_ultima_trans,pasv_saldo_total_1mes_atras,pasv_saldo_ca_1mes_atras,PERIODO,num_tx_ult_1mes_pasv,num_tx_ult_2meses_pasv,num_tx_ult_6meses_pasv,sum_tx_linea_pasv,sum_tx_nolinea_pasv,fecha_ult_tx,dias_desde_ult_tx_pasv,num_tx_ult_1mes_ahr,num_tx_ult_4meses_ahr,sum_tx_linea_ahr,sum_tx_nolinea_ahr,dias_desde_ult_tx_ahr,score_info_banco,decil_info_banco,grupo_prob_info_banco,decil_TU
0,1,10001108,Act_02_19_202007,44044,44135,CLIENTE ACTIVO,BOGOTA,205552310859223401,10001108,0.000330,2.0,grupo_prob_alta,grupo_mkd,seg_perfilado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000413,3,grupo_prob_alta,4.0
1,1,10003005,Act_02_19_202007,44044,44135,CLIENTE ACTIVO,GIRARDOT,767952295444029901,10003005,0.001942,1.0,grupo_prob_alta,grupo_mkd,seg_perfilado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,202006.0,153.266667,9.733333,11.0,71.567122,71.567122,202006,13.368421,28.210526,168.473684,17.0,NaN,2020-06-27,3.0,13.368421,76.947368,17.0,NaN,3.0,0.001110,4,grupo_prob_alta,5.0
2,1,10011983,Act_02_19_202007,44044,44135,CLIENTE ACTIVO,CALI,521152307006793601,10011983,0.000021,5.0,grupo_prob_baja,grupo_mkd,seg_perfilado,1.0,1000000.0,1000000.0,922319.0,0.0,7.82345,0.922319,6.0,173677.0,14.28,1.0,1.0,0.0,ORDEN NACIONAL,Policia Nacional,NACIONAL CENTRALIZADA,2020-07,202007.0,202006.0,193.466667,193.466667,8.0,1.042724,1.042724,202006,10.250000,19.250000,73.000000,16.0,NaN,2020-06-30,0.0,10.250000,42.250000,16.0,NaN,0.0,0.000480,3,grupo_prob_alta,1.0
3,1,10012781,Act_02_19_202007,44044,44135,CLIENTE ACTIVO,BOGOTA,158852311657703101,10012781,0.000099,3.0,grupo_prob_alta,grupo_mkd,seg_perfilado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,202006.0,78.600000,78.600000,8.0,0.088371,0.088371,202006,10.388889,19.333333,63.666667,18.0,NaN,2020-06-27,3.0,10.388889,38.722222,18.0,NaN,3.0,0.005875,5,grupo_prob_alta,3.0
4,1,10012969,Act_02_19_202007,44044,44135,CLIENTE ACTIVO,MEDELLIN,797552318205826201,10012969,0.000557,2.0,grupo_prob_alta,grupo_mkd,seg_perfilado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000413,3,grupo_prob_alta,4.0


In [29]:
data_finalF.count()

tipo_identificacion             13565
numero_identificacion           13565
nombre_campaña                  13565
fecha_inicio                    13565
fecha_fin                       13565
tipo                            13565
ciudad_crm                      13565
id_cliente                      13565
id_num_identificacion           13565
score                           13565
decil                           13565
grupo_prob                      13565
grupos                          13565
segmento                        13565
num_lib_solicitadas              1302
prom_monto_solicitado            1302
prom_monto_aprobado              1302
prom_monto_desembolsado          1302
prom_monto_novado                1302
min_anos_ult_lib                 1302
prom_prop_desembolso             1302
prom_n_cuotas                    1302
prom_valor_cuota                 1302
prom_tasa                        1302
prom_dias_procesamiento          1302
max_dias_procesamiento           1302
n_novaciones

In [30]:
## Comparación decil

pd.crosstab(data_finalF['decil_info_banco'],data_finalF['decil_TU'])


decil_TU,1.0,2.0,3.0,4.0,5.0
decil_info_banco,,,,,
1,804,623,485,460,399
2,470,371,290,308,279
3,1361,1641,1634,1765,1642
4,132,117,97,121,162
5,78,68,79,76,103


In [31]:
pd.crosstab(data_finalF['grupo_prob_info_banco'],data_finalF['grupo_prob'])

grupo_prob,grupo_prob_alta,grupo_prob_baja
grupo_prob_info_banco,,
grupo_prob_alta,5679,3397
grupo_prob_baja,2221,2268


In [32]:
## Guardando base
path = '/mnt/work/CU_adquisicionTC/objects/base_compar_scores_perf.csv'

data_finalF.to_csv(path, sep='|', encoding='utf-8')

In [35]:
data_finalF.head()

,tipo_identificacion,numero_identificacion,nombre_campaña,fecha_inicio,fecha_fin,tipo,ciudad_crm,id_cliente,id_num_identificacion,score,decil,grupo_prob,grupos,segmento,num_lib_solicitadas,prom_monto_solicitado,prom_monto_aprobado,prom_monto_desembolsado,prom_monto_novado,min_anos_ult_lib,prom_prop_desembolso,prom_n_cuotas,prom_valor_cuota,prom_tasa,prom_dias_procesamiento,max_dias_procesamiento,n_novaciones,sector_ultlibranza,subsector_ultlibranza,tipo_pagaduria_ultlibranza,periodo,fecha_lib,fecha_pasivo,pasv_antig_total,pasv_num_meses_ult_apertura,pasv_dias_desde_ultima_trans,pasv_saldo_total_1mes_atras,pasv_saldo_ca_1mes_atras,PERIODO,num_tx_ult_1mes_pasv,num_tx_ult_2meses_pasv,num_tx_ult_6meses_pasv,sum_tx_linea_pasv,sum_tx_nolinea_pasv,fecha_ult_tx,dias_desde_ult_tx_pasv,num_tx_ult_1mes_ahr,num_tx_ult_4meses_ahr,sum_tx_linea_ahr,sum_tx_nolinea_ahr,dias_desde_ult_tx_ahr,score_info_banco,decil_info_banco,grupo_prob_info_banco,decil_TU
0,1,10001108,Act_02_19_202007,44044,44135,CLIENTE ACTIVO,BOGOTA,205552310859223401,10001108,0.000330,2.0,grupo_prob_alta,grupo_mkd,seg_perfilado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000413,3,grupo_prob_alta,4.0
1,1,10003005,Act_02_19_202007,44044,44135,CLIENTE ACTIVO,GIRARDOT,767952295444029901,10003005,0.001942,1.0,grupo_prob_alta,grupo_mkd,seg_perfilado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,202006.0,153.266667,9.733333,11.0,71.567122,71.567122,202006,13.368421,28.210526,168.473684,17.0,NaN,2020-06-27,3.0,13.368421,76.947368,17.0,NaN,3.0,0.001110,4,grupo_prob_alta,5.0
2,1,10011983,Act_02_19_202007,44044,44135,CLIENTE ACTIVO,CALI,521152307006793601,10011983,0.000021,5.0,grupo_prob_baja,grupo_mkd,seg_perfilado,1.0,1000000.0,1000000.0,922319.0,0.0,7.82345,0.922319,6.0,173677.0,14.28,1.0,1.0,0.0,ORDEN NACIONAL,Policia Nacional,NACIONAL CENTRALIZADA,2020-07,202007.0,202006.0,193.466667,193.466667,8.0,1.042724,1.042724,202006,10.250000,19.250000,73.000000,16.0,NaN,2020-06-30,0.0,10.250000,42.250000,16.0,NaN,0.0,0.000480,3,grupo_prob_alta,1.0
3,1,10012781,Act_02_19_202007,44044,44135,CLIENTE ACTIVO,BOGOTA,158852311657703101,10012781,0.000099,3.0,grupo_prob_alta,grupo_mkd,seg_perfilado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,202006.0,78.600000,78.600000,8.0,0.088371,0.088371,202006,10.388889,19.333333,63.666667,18.0,NaN,2020-06-27,3.0,10.388889,38.722222,18.0,NaN,3.0,0.005875,5,grupo_prob_alta,3.0
4,1,10012969,Act_02_19_202007,44044,44135,CLIENTE ACTIVO,MEDELLIN,797552318205826201,10012969,0.000557,2.0,grupo_prob_alta,grupo_mkd,seg_perfilado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000413,3,grupo_prob_alta,4.0


In [40]:
data_finalF['numero_identificacion'] = data_finalF['numero_identificacion'].str.replace('nan','-999').fillna('-999').astype('int')

In [34]:
## Cargando base propensión:


path = 's3://adl-refined-dev-popular/data_orig/tmp/tmpg_Base_Propension_20201130.xlsx'

data_rpilot = pd.read_excel(path, sheet_name=0)
print(data_rpilot.shape)
data_rpilot.head()

(26423, 16)


,CC,CANAL,FECHA CARGUE,GESTION,CONTACTO,VENTA,VENTAS - # TC,ACTIVACION,# TARJETAS,PRIMER USO,CAMPAÑA,BASE ADL,Record_Count,TIPIFICACIÓN_1,TIPIFICACIÓN_2,CANAL 2
0,124033,Atento,2020-09-07,1,1,0,0,0,0,0,PERFILADOS,ALTA PROBABILIDAD,1,NO INTERESADO,Ya tiene otra tarjeta,OFICINAS & VENDOORS
1,130354,MKD,NaT,0,0,0,0,0,0,0,PERFILADO,BAJA PROBABILIDAD MKD,1,NaN,NaN,MKD
2,137820,Atento,2020-09-07,0,0,0,0,0,0,0,PERFILADOS,PERFILADOS CONTROL,1,NaN,NaN,OFICINAS & VENDOORS
3,162580,Project,2020-09-07,1,1,0,0,0,0,0,PERFILADOS,PERFILADOS CONTROL,1,No desea mas deudas LD/TDC,NaN,OFICINAS & VENDOORS
4,176109,MKD,NaT,0,0,0,0,0,0,0,PREAPROBADO,BAJA PROBABILIDAD MKD,1,AUT - Silencio,NaN,MKD


In [39]:
data_finalF.dtypes

tipo_identificacion               int64
numero_identificacion            object
nombre_campaña                   object
fecha_inicio                      int64
fecha_fin                         int64
tipo                             object
ciudad_crm                       object
id_cliente                        int64
id_num_identificacion            object
score                           float32
decil                           float64
grupo_prob                       object
grupos                           object
segmento                         object
num_lib_solicitadas             float64
prom_monto_solicitado           float32
prom_monto_aprobado             float32
prom_monto_desembolsado         float32
prom_monto_novado               float32
min_anos_ult_lib                float64
prom_prop_desembolso            float32
prom_n_cuotas                   float64
prom_valor_cuota                float32
prom_tasa                       float32
prom_dias_procesamiento         float64


In [36]:
data_rpilot.columns = data_rpilot.columns.str.lower()
data_rpilot = data_rpilot.rename(columns={'cc':'numero_identificacion'})
data_rpilot.columns

Index(['numero_identificacion', 'canal', 'fecha cargue', 'gestion', 'contacto',
       'venta', 'ventas - # tc', 'activacion', '# tarjetas', 'primer uso',
       'campaña', 'base adl', 'record_count', 'tipificación_1',
       'tipificación_2', 'canal 2'],
      dtype='object')

In [41]:
data_fin = pd.merge(data_finalF,data_rpilot[['numero_identificacion', 'canal', 'fecha cargue', 'gestion', 'contacto',
       'venta', 'ventas - # tc', 'activacion', '# tarjetas', 'primer uso',
       'campaña', 'base adl', 'record_count', 'tipificación_1',
       'tipificación_2', 'canal 2']], on = 'numero_identificacion', how='left')

data_fin.head()

,tipo_identificacion,numero_identificacion,nombre_campaña,fecha_inicio,fecha_fin,tipo,ciudad_crm,id_cliente,id_num_identificacion,score,decil,grupo_prob,grupos,segmento,num_lib_solicitadas,prom_monto_solicitado,prom_monto_aprobado,prom_monto_desembolsado,prom_monto_novado,min_anos_ult_lib,prom_prop_desembolso,prom_n_cuotas,prom_valor_cuota,prom_tasa,prom_dias_procesamiento,max_dias_procesamiento,n_novaciones,sector_ultlibranza,subsector_ultlibranza,tipo_pagaduria_ultlibranza,periodo,fecha_lib,fecha_pasivo,pasv_antig_total,pasv_num_meses_ult_apertura,pasv_dias_desde_ultima_trans,pasv_saldo_total_1mes_atras,pasv_saldo_ca_1mes_atras,PERIODO,num_tx_ult_1mes_pasv,num_tx_ult_2meses_pasv,num_tx_ult_6meses_pasv,sum_tx_linea_pasv,sum_tx_nolinea_pasv,fecha_ult_tx,dias_desde_ult_tx_pasv,num_tx_ult_1mes_ahr,num_tx_ult_4meses_ahr,sum_tx_linea_ahr,sum_tx_nolinea_ahr,dias_desde_ult_tx_ahr,score_info_banco,decil_info_banco,grupo_prob_info_banco,decil_TU,canal,fecha cargue,gestion,contacto,venta,ventas - # tc,activacion,# tarjetas,primer uso,campaña,base adl,record_count,tipificación_1,tipificación_2,canal 2
0,1,10001108,Act_02_19_202007,44044,44135,CLIENTE ACTIVO,BOGOTA,205552310859223401,10001108,0.000330,2.0,grupo_prob_alta,grupo_mkd,seg_perfilado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000413,3,grupo_prob_alta,4.0,MKD,NaT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,PERFILADO,ALTA PROBABILIDAD MKD,1.0,NaN,NaN,MKD
1,1,10003005,Act_02_19_202007,44044,44135,CLIENTE ACTIVO,GIRARDOT,767952295444029901,10003005,0.001942,1.0,grupo_prob_alta,grupo_mkd,seg_perfilado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,202006.0,153.266667,9.733333,11.0,71.567122,71.567122,202006,13.368421,28.210526,168.473684,17.0,NaN,2020-06-27,3.0,13.368421,76.947368,17.0,NaN,3.0,0.001110,4,grupo_prob_alta,5.0,MKD,NaT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,PERFILADO,ALTA PROBABILIDAD MKD,1.0,NaN,NaN,MKD
2,1,10011983,Act_02_19_202007,44044,44135,CLIENTE ACTIVO,CALI,521152307006793601,10011983,0.000021,5.0,grupo_prob_baja,grupo_mkd,seg_perfilado,1.0,1000000.0,1000000.0,922319.0,0.0,7.82345,0.922319,6.0,173677.0,14.28,1.0,1.0,0.0,ORDEN NACIONAL,Policia Nacional,NACIONAL CENTRALIZADA,2020-07,202007.0,202006.0,193.466667,193.466667,8.0,1.042724,1.042724,202006,10.250000,19.250000,73.000000,16.0,NaN,2020-06-30,0.0,10.250000,42.250000,16.0,NaN,0.0,0.000480,3,grupo_prob_alta,1.0,MKD,NaT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,PERFILADO,BAJA PROBABILIDAD MKD,1.0,NaN,NaN,MKD
3,1,10012781,Act_02_19_202007,44044,44135,CLIENTE ACTIVO,BOGOTA,158852311657703101,10012781,0.000099,3.0,grupo_prob_alta,grupo_mkd,seg_perfilado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,202006.0,78.600000,78.600000,8.0,0.088371,0.088371,202006,10.388889,19.333333,63.666667,18.0,NaN,2020-06-27,3.0,10.388889,38.722222,18.0,NaN,3.0,0.005875,5,grupo_prob_alta,3.0,MKD,NaT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,PERFILADO,ALTA PROBABILIDAD MKD,1.0,NaN,NaN,MKD
4,1,10012969,Act_02_19_202007,44044,44135,CLIENTE ACTIVO,MEDELLIN,797552318205826201,10012969,0.000557,2.0,grupo_prob_alta,grupo_mkd,seg_perfilado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000413,3,grupo_prob_alta,4.0,MKD,NaT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,PERFILADO,ALTA PROBABILIDAD MKD,1.0,NaN,NaN,MKD


In [44]:
data_fin['campaña'].value_counts()

PERFILADOS    7017
PERFILADO     6021
Name: campaña, dtype: int64

In [45]:
pd.crosstab(data_fin['base adl'],data_fin['grupo_prob'])

grupo_prob,grupo_prob_alta,grupo_prob_baja
base adl,,
ALTA PROBABILIDAD,2113,0
ALTA PROBABILIDAD MKD,3525,0
BAJA PROBABILIDAD,0,1560
BAJA PROBABILIDAD MKD,0,2496
PERFILADOS CONTROL,1985,1359


In [42]:
data_fin.count()

tipo_identificacion             13569
numero_identificacion           13569
nombre_campaña                  13569
fecha_inicio                    13569
fecha_fin                       13569
tipo                            13569
ciudad_crm                      13569
id_cliente                      13569
id_num_identificacion           13569
score                           13569
decil                           13569
grupo_prob                      13569
grupos                          13569
segmento                        13569
num_lib_solicitadas              1302
prom_monto_solicitado            1302
prom_monto_aprobado              1302
prom_monto_desembolsado          1302
prom_monto_novado                1302
min_anos_ult_lib                 1302
prom_prop_desembolso             1302
prom_n_cuotas                    1302
prom_valor_cuota                 1302
prom_tasa                        1302
prom_dias_procesamiento          1302
max_dias_procesamiento           1302
n_novaciones

In [46]:
## Guardando base
path = '/mnt/work/CU_adquisicionTC/objects/base_compar_scores_perf.csv'

data_fin.to_csv(path, sep='|', encoding='utf-8')